In [2]:
# === Step 1: Import necessary libraries ===

# Import the pandas library for data manipulation in DataFrame format, which is ideal for time series.
import pandas as pd

# Import the numpy library, which is commonly used for numerical calculations, especially with arrays.
import numpy as np

# Import the yfinance library to fetch financial data, such as closing prices for ETFs.
import yfinance as yf

# Import the datetime module and its components (datetime, date, timedelta) for managing dates and time periods.
from datetime import datetime, date, timedelta


import warnings
warnings.filterwarnings('ignore')  # Suppress all warnings

# === Step 2: Download 10 years of daily data ===

# Define a list with the symbols (tickers) of the ETFs we will use in the analysis.

tickers = [
    "SPY",  # US Stocks - represents the S&P 500 index
    "AGG",  # US Bonds - represents investment-grade US bonds
    "TLT",  # Long-term Bonds - represents long-term US government bonds (20+ years)
    "GLD",  # Gold - represents the price of gold
    "DBC",  # Commodities - represents a broad range of commodities
    "IEF",  # Intermediate Bonds - represents intermediate-term US government bonds (7-10 years)
    "BIL",  # Short-term Bonds - represents short-term US government bonds (1-3 years)
    "VBR",  # Small-cap value stocks - represents small-cap stocks with value characteristics
    "VV",   # Large-cap stocks - represents US large-cap stocks]
]

# Calculate the start date for data retrieval, subtracting 10 years from today's date.

start_date = date.today() - timedelta(days=365*10)

# Define the end date for data retrieval as today's date.

end_date = date.today()

# Use the yf.download() function to download daily closing price data ('Close').
# auto_adjust=True ensures that the prices are adjusted for splits and dividends.
# Select only the 'Close' column to work with closing prices.

data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)["Close"]

# Calculate daily percentage returns from closing prices.
# The .pct_change() method calculates the percentage change between consecutive days.

df_daily_returns = data.pct_change()

# Create a DataFrame with the same index and columns as df_daily_returns.
# This DataFrame will store the cumulative values of the assets.

df_compound_values = pd.DataFrame(index=df_daily_returns.index, columns=df_daily_returns.columns)

# Set the first row (iloc[0]) to 1.
# This represents the initial value (e.g., 1 unit) of each asset on the first day.

df_compound_values.iloc[0] = 1

# Calculate the cumulative product of (1 + daily_returns).
# This gives us the cumulative return for each asset from the second day onwards.
# We start from iloc[1:] because iloc[0] has already been set to 1.

df_compound_values.iloc[1:] = (1 + df_daily_returns.iloc[1:]).cumprod()


# === Step 3: Define portfolios and weights ===
# Define the different portfolios as a dictionary.
# Each dictionary key is the portfolio name (e.g., "60_40").
# Each value is another dictionary containing the ETFs (tickers) and their corresponding static weights in that portfolio.

portfolios = {
    "60_40": {"SPY": 0.60, "AGG": 0.40},
    "All_Weather": {"SPY": 0.30, "TLT": 0.40, "IEF": 0.15, "GLD": 0.075, "DBC": 0.075},
    "Permanent": {"SPY": 0.25, "TLT": 0.25, "GLD": 0.25, "BIL": 0.25},
    "Golden_Butterfly": {"SPY": 0.20, "VBR": 0.20, "VV": 0.20, "TLT": 0.20, "GLD": 0.20},
}


# === Step 4: Function to calculate portfolio returns with rebalancing ===

def portfolio_rebalanced_returns(portfolio: dict[str, float], df_daily_returns: pd.DataFrame, rebalance_period: int = 21) -> pd.Series:
    """
    Calculates the cumulative return of a portfolio with periodic rebalancing.

    Args:
     portfolio (dict): A dictionary where keys are the asset tickers (str)
                       and values are their corresponding weights (float) in the portfolio.
     df_daily_returns (pd.DataFrame): A DataFrame containing the daily returns
                                    of the assets. The index should be the date,
                                    and columns should be the asset tickers.
     rebalance_period (int, optional): The number of trading days between rebalancing.
                                      The default value is 21, which corresponds to approximately one month.

    Returns:
     pd.Series: A Series containing the cumulative value of the portfolio
                over time. The index is the date.
    """
    # Create a pandas Series to store the total portfolio value
    # for each date.
    portfolio_value = pd.Series(index=data.index, dtype=float)

    # Initialize the portfolio value on the first day (index 0) to 1.0.
    # This serves as a reference point, where an initial investment of 1 unit is tracked over time.
    portfolio_value.iloc[0] = 1.0

    # Set the portfolio value to 0.0 for all other days initially.
    portfolio_value[1:] = 0.0

    # Create a pandas DataFrame to store the value of each asset (ETF)
    # separately within the portfolio, for each date.

    asset_value = pd.DataFrame(index=data.index, dtype=float)

    # Initialize the value of each asset on the first day (index 0).
    # The initial value of each asset is equal to its initial weight in the portfolio.
    # This means that from the initial portfolio value (1.0), the percentage portfolio[ticker]
    # was invested in this asset.

    for ticker in portfolio:
        # Create a column for each ticker in the asset_value DataFrame.
        asset_value[ticker] = pd.Series(index=data.index, dtype=float)

        # Set the initial value of the asset on the first day equal to its weight in the portfolio.
        asset_value[ticker].iloc[0] = portfolio[ticker]

    # Iterate through each day in the data history, starting from the second day (index 1).
    for i in range(1, len(data)):
        # Check if the previous day (i-1) was a rebalancing day.
        # The condition (i - 1) % rebalance_period == 0 checks if the number of days
        # that have passed since the start is a multiple of the rebalancing period.
        # If it is a rebalancing day (based on the previous day), the asset allocation
        # for the current day (i) is done based on the initial weights.
        if (i - 1) % rebalance_period == 0:
            # === Rebalancing Day ===
            # On a rebalancing day, the value of each asset is recalculated
            # based on its initial weight, the total portfolio value
            # at the end of the previous day, and the asset's daily return
            # on the current day.
            # The total portfolio value for the current day will be calculated
            # by summing the new asset values.
            for ticker in portfolio:
                # The new value of asset[ticker] on day 'i' is:
                # (the initial weight of the ticker) * (the total portfolio value on the previous day) * (1 + the daily return of the ticker on day 'i').
                asset_value[ticker].iloc[i] = portfolio[ticker] * portfolio_value.iloc[i - 1] * (1 + df_daily_returns[ticker].iloc[i])

                # Add the calculated new value of asset[ticker] to the total value
                # of the portfolio for the current day.
                portfolio_value.iloc[i] += asset_value[ticker].iloc[i]
        else:
            # === Non-Rebalancing Day ===
            # On a day that is not a rebalancing day, the value of each asset simply
            # updates based on its value on the previous day and its daily return.
            # The total portfolio value for the current day will be calculated
            # by summing the updated asset values.
            for ticker in portfolio:
                # The new value of asset[ticker] on day 'i' is:
                # (the value of asset[ticker] on the previous day) * (1 + the daily return of the ticker on day 'i').
                asset_value[ticker].iloc[i] = asset_value[ticker].iloc[i - 1] * (1 + df_daily_returns[ticker].iloc[i])

                # Add the calculated new value of asset[ticker] to the total value
                # of the portfolio for the current day.
                portfolio_value.iloc[i] += asset_value[ticker].iloc[i]

    # Return the Series containing the cumulative portfolio value over time.
    return portfolio_value


# === Step 5: Calculate value for each predefined portfolio ===
# Create a pandas DataFrame to store the cumulative values (returns)
# for each of the predefined portfolios.

df_portfolios_values = pd.DataFrame()

# Iterate through each portfolio definition in 'portfolios' (Step 3).
# .items() gives us both the name (key) and the weights dictionary (value) for each portfolio.
for name, portfolio in portfolios.items():
    # For each portfolio, call the portfolio_rebalanced_returns function
    # passing it the weights dictionary and the DataFrame with daily returns.
    # The result (a Series with the portfolio value time series)
    # is stored as a new column in the df_portfolios_values DataFrame,
    # using the portfolio name as the column name.
    df_portfolios_values[name] = portfolio_rebalanced_returns(portfolio, df_daily_returns)


# === Step 6: Display results ===
# Display the final DataFrame containing the time series of cumulative values
# for all portfolios.
# display(df_portfolios_values) # Display is commented out here, as the following cells
#                                # can be used for displaying or
#                                # further analysis of df_portfolios_values.

[*********************100%***********************]  9 of 9 completed


In [ ]:
# === Step 7: Save results to Database and CSV ===
import sqlite3

# Reset the DataFrame's index (Date) to a regular column ('Date').
# This is necessary for saving to a database or CSV file,
# where the date is usually a data column.
df_values = df_portfolios_values.reset_index()  # The 'Date' column is created from the previous index.

# Calculate daily returns for each portfolio.
# The .pct_change() method calculates the percentage change from the previous day.
# Then, we use .stack() to convert the DataFrame from wide format (each column a portfolio)
# to long format (one column for portfolio names and one column for returns).
# .reset_index() converts the multilevel index created by stack into regular columns.
df_portfolio_returns = df_portfolios_values.pct_change().stack().reset_index()
# Rename the columns of the new DataFrame for clarity.
# The columns will be: date, portfolio name, and daily return.
df_portfolio_returns.columns = ['Date','name','daily_return']
# Similarly for tickers
df_ticker_returns = df_daily_returns.stack().reset_index()
df_ticker_returns.columns = ['Date','ticker','daily_return']

df_ticker_values = df_compound_values.stack().reset_index()
df_ticker_values.columns = ['Date','ticker','value']
# === Save to SQLite database ===
# Create a connection to the SQLite database file named 'portfolio_data.db'.
# If the file does not exist, it will be created.
conn = sqlite3.connect("portfolio_data.db")
# Save the DataFrame with cumulative values ('df_values') to an SQLite table named 'portfolio_values'.
# if_exists='replace': If the table already exists, it will be replaced.
# index=False: Do not save the DataFrame index as a column in the table.
df_values.to_sql('portfolio_values', conn, if_exists='replace', index=False)   # cumulative values
# Save the DataFrame with daily returns ('df_portfolio_returns') to an SQLite table named 'portfolio_returns'.
# Use the same options for replacement and index.
df_portfolio_returns.to_sql('portfolio_returns', conn, if_exists='replace', index=False)  # daily returns

# Save raw ETF data along with their returns and values
data.reset_index().to_sql('etf_prices', conn, if_exists='replace', index=False)
df_ticker_returns.to_sql('etf_returns', conn, if_exists='replace', index=False)
df_ticker_values.to_sql('etf_values', conn, if_exists='replace', index=False)
# Close the database connection to release resources.
conn.close()

# === Export to CSV files (e.g., for use in Power BI) ===
# Save the DataFrame with cumulative values to a CSV file named "portfolio_values.csv".
# index=False: Do not write the DataFrame index to the CSV file.
df_values.to_csv("portfolio_values.csv", index=False)
# Save the DataFrame with daily returns to a CSV file named "portfolio_returns.csv".
df_portfolio_returns.to_csv("portfolio_returns.csv", index=False)

# Similarly for raw data, their returns, and their values
df_ticker_returns.to_csv("etf_returns.csv", index=False)
df_ticker_values.to_csv("etf_values.csv", index=False)
data.reset_index().to_csv("etf_prices.csv", index=False)
# === Print information for confirmation ===
# Print a confirmation message that the files have been saved.
print("Saved raw etf prices their daily returns, portfolio_values and portfolio_returns to DB and CSV.")

Saved raw etf prices their daily returns, portfolio_values and portfolio_returns to DB and CSV.


In [ ]:
# === Calculate simple metrics (KPIs) for Portfolios and Tickers ===
# Import the numpy library for numerical calculations, such as calculating the square root.
import numpy as np
# Import the sqlite3 library for connecting and interacting with the SQLite database.
import sqlite3
# Import the pandas library for processing data in DataFrame and Series format.
import pandas as pd

# Define the number of trading days in a year.
# This constant is used for the annual calculation of volatility and average return.
TRADING_DAYS = 252
# Define the annual return of the "risk-free" asset.
# This value is used in the calculation of the Sharpe ratio.
RISK_FREE = 0.01   # 1% annual

# Create an empty list where we will store the calculated metrics for each portfolio and ticker.
metrics = []

# === Calculate metrics for Portfolios ===
# Iterate over each column (i.e., for each portfolio) in the DataFrame 'df_portfolios_values'.
# df_portfolios_values contains the time series of the cumulative value for each portfolio.
for name in df_portfolios_values.columns:
    # Select the Series corresponding to the current portfolio ('name').
    series = df_portfolios_values[name].sort_index()
    # Define the Series as 'wealth' for easier readability. This represents the cumulative value of the portfolio.
    wealth = series  # cumulative value (starts around 1)
    # Calculate the total return of the portfolio from the beginning to the end of the period.
    # It is the final value minus the initial value (which is 1.0).
    total_return = wealth.iloc[-1] - 1
    # Calculate the number of days between the first and last date of the data.
    days = (wealth.index[-1] - wealth.index[0]).days
    # Calculate the number of years. We use 365.25 to account for leap years.
    # If the number of days is 0, the value is NaN (Not a Number) to avoid division by zero.
    years = days/365.25 if days>0 else np.nan
    # Calculate the Compound Annual Growth Rate (CAGR).
    # We use the formula CAGR = (Final Value / Initial Value) ^ (1 / Number of Years) - 1.
    # If the number of years is 0, the value is NaN.
    CAGR = (wealth.iloc[-1]) ** (1/years) - 1 if years>0 else np.nan
    # Calculate the daily returns of the portfolio.
    # We use .pct_change() on the 'series' (which is 'wealth').
    # .dropna() removes the first value which is NaN (since there is no previous day for the first return).
    daily_rets = series.pct_change().dropna()
    # Calculate the annual volatility.
    # It is the standard deviation of the daily returns multiplied by the square root of the number of trading days.
    vol = daily_rets.std() * np.sqrt(TRADING_DAYS)
    # Calculate the mean annual return.
    # It is the average daily return multiplied by the number of trading days.
    mean_ann = daily_rets.mean() * TRADING_DAYS
    # Calculate the Sharpe ratio.
    # The formula is (Mean Annual Return - Risk-Free Rate) / Annual Volatility.
    # If the volatility is 0, the value is NaN.
    sharpe = (mean_ann - RISK_FREE) / vol if vol>0 else np.nan
    # Calculate the peak value of the portfolio up to each specific date.
    # .cummax() calculates the cumulative maximum.
    peak = wealth.cummax()
    # Calculate the drawdown. It is the percentage drop from the previous peak.
    # The formula is (Current Value - Peak) / Peak.
    drawdown = (wealth - peak) / peak
    # Calculate the maximum drawdown (Max Drawdown - MaxDD).
    # It is the largest percentage drop from a peak.
    maxdd = drawdown.min()
    # Create a dictionary with the calculated metrics for the current portfolio.
    metrics.append({'name':name, 'CAGR':CAGR, 'AnnualVol':vol, 'Sharpe':sharpe, 'MaxDrawdown':maxdd, 'TotalReturn':total_return, 'Type': 'Portfolio'})

# === Calculate metrics for Tickers ===
# Iterate over each column (i.e., for each ticker) in the DataFrame 'df_compound_values'.
# df_compound_values contains the time series of the cumulative value for each ticker.
for etf in df_compound_values.columns:
    # Select the Series corresponding to the current ticker.
    series = df_compound_values[etf].sort_index()
    # Define the Series as 'wealth'. This represents the cumulative value of the ticker.
    wealth = series
    # Calculate the total return of the ticker from the beginning to the end of the period.
    # It is the final value minus the initial value (which is 1.0).
    total_return = wealth.iloc[-1] - 1
    # Calculate the number of days between the first and last date of the data.
    days = (wealth.index[-1] - wealth.index[0]).days
    # Calculate the number of years.
    years = days/365.25 if days>0 else np.nan
    # Calculate the Compound Annual Growth Rate (CAGR).
    CAGR = (wealth.iloc[-1]) ** (1/years) - 1 if years>0 else np.nan
    # Calculate the daily returns of the ticker.
    # We use .pct_change() on the 'series'.
    # .dropna() removes the first value which is NaN.
    daily_rets = series.pct_change().dropna()
    # Calculate the annual volatility.
    vol = daily_rets.std() * np.sqrt(TRADING_DAYS)
    # Calculate the mean annual return.
    mean_ann = daily_rets.mean() * TRADING_DAYS
    # Calculate the Sharpe ratio.
    sharpe = (mean_ann - RISK_FREE) / vol if vol>0 else np.nan
    # Calculate the peak value of the ticker.
    peak = wealth.cummax()
    # Calculate the drawdown.
    drawdown = (wealth - peak) / peak
    # Calculate the maximum drawdown (MaxDD).
    maxdd = drawdown.min()
    # Create a dictionary with the calculated metrics for the current ticker.
    metrics.append({'name':etf, 'CAGR':CAGR, 'AnnualVol':vol, 'Sharpe':sharpe, 'MaxDrawdown':maxdd, 'TotalReturn':total_return, 'Type': 'Ticker'})


# Convert the list of dictionaries into a pandas DataFrame.
df_metrics = pd.DataFrame(metrics).sort_values('CAGR', ascending=False)

# === Save metrics ===
# Save the DataFrame with the metrics to a CSV file named "portfolio_metrics.csv".
# index=False: Do not write the DataFrame index to the CSV file.
df_metrics.to_csv("portfolio_metrics.csv", index=False)
# Create a connection to the SQLite database file.
conn = sqlite3.connect("portfolio_data.db")
# Save the DataFrame with the metrics to an SQLite table named 'portfolio_metrics'.
# if_exists='replace': If the table already exists, it will be replaced.
# index=False: Do not save the DataFrame index as a column in the table.
df_metrics.to_sql('portfolio_metrics', conn, if_exists='replace', index=False)
# Close the database connection.
conn.close()

# Print a confirmation message and display the DataFrame with the metrics.
print("Saved portfolio_metrics.csv and wrote portfolio_metrics table to DB")
print(df_metrics)

Saved portfolio_metrics.csv and wrote portfolio_metrics table to DB
                name      CAGR  AnnualVol    Sharpe  MaxDrawdown  TotalReturn  \
12                VV  0.147281   0.183564  0.788127    -0.342791     2.942768   
9                SPY  0.146815   0.181055  0.794209    -0.337173     2.926798   
7                GLD  0.121852   0.144769  0.799667    -0.220022     2.152137   
3   Golden_Butterfly  0.109063   0.115897  0.867289    -0.220866     1.811169   
11               VBR  0.099925   0.218679  0.500964    -0.452752     1.588271   
0              60_40  0.096970   0.111259  0.799899    -0.214679     1.519654   
2          Permanent  0.075028   0.071201  0.913883    -0.174904     1.059323   
1        All_Weather  0.062019   0.086722  0.623725    -0.235363     0.823608   
6                DBC  0.053804   0.176830  0.329393    -0.417111     0.687548   
5                BIL  0.019141   0.002632  3.423261    -0.002085     0.208425   
4                AGG  0.018949   0.053843

/tmp/ipython-input-2120807102.py:85: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  daily_rets = series.pct_change().dropna()


In [ ]:
conn = sqlite3.connect("portfolio_data.db")
Average_cagr = pd.read_sql_query("""
SELECT PRINTF('%.2f%%', AVG(CAGR) * 100) AS AVERAGE_CAGR
        FROM portfolio_metrics
       WHERE name IN ("SPY", "Golden_Butterfly", "60_40", "All_Weather", "Permanent")

""", conn)
TopSharpe = pd.read_sql_query("""
SELECT name AS Best_Sharpe_Ratio , Sharpe
        FROM portfolio_metrics
       WHERE name IN ("SPY", "Golden_Butterfly", "60_40", "All_Weather", "Permanent")
       ORDER BY Sharpe DESC
       LIMIT 1
""", conn)
Maxdd=pd.read_sql_query("""
SELECT name AS Worst_Drawdown , MaxDrawdown
        FROM portfolio_metrics
       WHERE name IN ("SPY", "Golden_Butterfly", "60_40", "All_Weather", "Permanent")
       ORDER BY MaxDrawdown ASC
       LIMIT 1
""", conn)


permanent_portfolio_trend = pd.read_sql_query("""
SELECT
    Date,
    Permanent
FROM (
    SELECT
        Date,
        Permanent
    FROM
        portfolio_values
    ORDER BY
        Date DESC
    LIMIT 756
) AS last_756_rows
ORDER BY
    Date ASC;
""", conn)

conn.close()
display(Average_cagr)
display(TopSharpe)
display(Maxdd)
display(permanent_portfolio_trend)

,AVERAGE_CAGR
0,9.80%


,Best_Sharpe_Ratio,Sharpe
0,Permanent,0.913883


,Worst_Drawdown,MaxDrawdown
0,SPY,-0.337173


,Date,Permanent
0,2022-09-01 00:00:00,1.446091
1,2022-09-02 00:00:00,1.447657
2,2022-09-06 00:00:00,1.435310
3,2022-09-07 00:00:00,1.450760
4,2022-09-08 00:00:00,1.447277
...,...,...
751,2025-09-02 00:00:00,2.013719
752,2025-09-03 00:00:00,2.026004
753,2025-09-04 00:00:00,2.031769
754,2025-09-05 00:00:00,2.045155
